Flight delays can be caused by a variety of factors, including weather, air traffic control issues, and mechanical problems. <br>Understanding these factors is crucial for developing strategies to mitigate delays and improve overall efficiency in air travel.

Import the necessary libraries:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import f_oneway
from scipy.stats import ttest_ind

Read the dataset:

In [2]:
df = pd.read_csv('./flight_data_2024_sample.csv')

Explore the dataset:

In [3]:
df.head()

,year,month,day_of_month,day_of_week,fl_date,op_unique_carrier,op_carrier_fl_num,origin,origin_city_name,origin_state_nm,...,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2024,4,18,4,2024-04-18,MQ,3535.0,DFW,"Dallas/Fort Worth, TX",Texas,...,0,151.0,144.0,119.0,835.0,0,0,0,0,0
1,2024,1,1,1,2024-01-01,AA,148.0,CLT,"Charlotte, NC",North Carolina,...,0,286.0,273.0,253.0,1773.0,0,0,0,0,0
2,2024,12,12,4,2024-12-12,9E,5440.0,CHA,"Chattanooga, TN",Tennessee,...,0,59.0,50.0,29.0,106.0,0,0,0,0,0
3,2024,4,8,1,2024-04-08,WN,1971.0,OMA,"Omaha, NE",Nebraska,...,0,180.0,177.0,163.0,1099.0,0,0,0,0,0
4,2024,2,16,5,2024-02-16,WN,862.0,BWI,"Baltimore, MD",Maryland,...,0,90.0,96.0,76.0,399.0,0,0,0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 10000 non-null  int64  
 1   month                10000 non-null  int64  
 2   day_of_month         10000 non-null  int64  
 3   day_of_week          10000 non-null  int64  
 4   fl_date              10000 non-null  object 
 5   op_unique_carrier    10000 non-null  object 
 6   op_carrier_fl_num    10000 non-null  float64
 7   origin               10000 non-null  object 
 8   origin_city_name     10000 non-null  object 
 9   origin_state_nm      10000 non-null  object 
 10  dest                 10000 non-null  object 
 11  dest_city_name       10000 non-null  object 
 12  dest_state_nm        10000 non-null  object 
 13  crs_dep_time         10000 non-null  int64  
 14  dep_time             9884 non-null   float64
 15  dep_delay            9884 non-null   

In [5]:
df.isnull().sum()

year                      0
month                     0
day_of_month              0
day_of_week               0
fl_date                   0
op_unique_carrier         0
op_carrier_fl_num         0
origin                    0
origin_city_name          0
origin_state_nm           0
dest                      0
dest_city_name            0
dest_state_nm             0
crs_dep_time              0
dep_time                116
dep_delay               116
taxi_out                120
wheels_off              120
wheels_on               127
taxi_in                 127
crs_arr_time              0
arr_time                127
arr_delay               164
cancelled                 0
cancellation_code      9878
diverted                  0
crs_elapsed_time          0
actual_elapsed_time     164
air_time                164
distance                  0
carrier_delay             0
weather_delay             0
nas_delay                 0
security_delay            0
late_aircraft_delay       0
dtype: int64

Drop the nulls from the dataset:

In [6]:
df_cleaned = df.fillna(0)


In [7]:
df_cleaned.isnull().sum()

year                   0
month                  0
day_of_month           0
day_of_week            0
fl_date                0
op_unique_carrier      0
op_carrier_fl_num      0
origin                 0
origin_city_name       0
origin_state_nm        0
dest                   0
dest_city_name         0
dest_state_nm          0
crs_dep_time           0
dep_time               0
dep_delay              0
taxi_out               0
wheels_off             0
wheels_on              0
taxi_in                0
crs_arr_time           0
arr_time               0
arr_delay              0
cancelled              0
cancellation_code      0
diverted               0
crs_elapsed_time       0
actual_elapsed_time    0
air_time               0
distance               0
carrier_delay          0
weather_delay          0
nas_delay              0
security_delay         0
late_aircraft_delay    0
dtype: int64

In [8]:
df_cleaned.shape

(10000, 35)

Find the mean of the 'ARRIVAL_DELAY' by day of the week in minutes:

In [9]:
df_delayed_mean = df_cleaned.groupby('day_of_week')['arr_delay'].mean().reset_index()
print(df_delayed_mean)

   day_of_week  arr_delay
0            1   6.320051
1            2   3.039286
2            3   4.560182
3            4   8.710696
4            5   9.820671
5            6   9.410769
6            7   9.928369


# ANOVA (Analysis of Variance)
Purpose:
ANOVA tests whether the means of several groups are equal.

Null hypothesis (H₀): The mean arrival delay is the same across all days of the week.

Alternative hypothesis (H₁): At least one day has a different mean arrival delay.

In [10]:
# Assume you have raw delays not just means, group them by day_of_week:
grouped_delays = [df_cleaned[df_cleaned['day_of_week'] == day]['arr_delay'] for day in df_cleaned['day_of_week'].unique()]

# One-way ANOVA
f_stat, p_value = f_oneway(*grouped_delays)
print(f"F-statistic: {f_stat}, p-value: {p_value}")

F-statistic: 3.5184680508912796, p-value: 0.0017655190007334887


p-value < 0.05, therefore we reject H₀: at least one group mean is significantly different.

# t-test
Purpose:
A t-test compares the means of two groups. Example: Compare mean delays for weekdays vs. weekends.

In [11]:
weekdays = df_cleaned[df_cleaned['day_of_week'].isin([1, 2, 3, 4, 5])]['arr_delay']
weekends = df_cleaned[df_cleaned['day_of_week'].isin([6, 7])]['arr_delay']

# Independent t-test
t_stat, p_value = ttest_ind(weekdays, weekends)
print(f"T-statistic: {t_stat}, p-value: {p_value}")

T-statistic: -2.4887031779056694, p-value: 0.012837141376664533


p-value < 0.05, therefore we reject H₀: there is a significant difference in mean delays between weekdays and weekends.

## Key Points on Interpreting Results
p-value: The probability that observed differences are due to chance.

Threshold (α): Commonly 0.05.

Reject H₀: If p-value < α, the difference is statistically significant.

Fail to Reject H₀: If p-value ≥ α, you don’t have enough evidence for a difference.